## Quick Start: Training and IMDb sentiment model with ULMFIT 

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.text import *

In [3]:
path = untar_data(URLs.IMDB_SAMPLE)

In [4]:
path.ls()

[PosixPath('/home/ricky/.fastai/data/imdb_sample/models'),
 PosixPath('/home/ricky/.fastai/data/imdb_sample/texts.csv'),
 PosixPath('/home/ricky/.fastai/data/imdb_sample/data_lm_export.pkl'),
 PosixPath('/home/ricky/.fastai/data/imdb_sample/data_clas_export.pkl')]

In [5]:
df = pd.read_csv(path/"texts.csv")
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [7]:
bs = 16

In [8]:
data_lm = TextLMDataBunch.from_csv(path, 'texts.csv')
data_clas = TextClasDataBunch.from_csv(path, 'texts.csv',
                                       vocab=data_lm.train_ds.vocab, bs=bs)

In [9]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [10]:
data_lm = load_data(path, 'data_lm_export.pkl', bs=bs)
data_clas = load_data(path, 'data_clas_export.pkl', bs=bs)

In [11]:
data_lm.show_batch()

idx,text
0,"xxmaj porter is for xxmaj will hay . xxmaj if you like old dark house movies and trains , then this is definitely for you . \n \n xxmaj strangely enough it 's the kind of film that you 'll want to see again and again . xxmaj it 's friendly and charming in an endearing sort of way with all of the nostalgic references that made great wartime"
1,"the government has transformed men into resources . xxmaj xxunk , resources that know xxmaj xxunk xxmaj xxunk and can make weapons out of anything . \n \n xxmaj sidenote : it 's always bothered me that , despite being a xxunk and hunted by the xxup cia , xxmaj interpol , and the police nearly xxup anywhere he goes , that xxmaj bourne never thought to make even"
2,"film . i ca n't decide which one that would be , so i say judge for xxunk . i do n't even know if renting it would be a good idea , the cost and all ... \n \n xxmaj plot : 0 / 10 xxmaj characters : 1 / 10 xxmaj acting : 2 / 10 xxmaj overall : 3 / 10 i feel like that 's"
3,"be different than his contemporaries , xxmaj lam gives us to traditional xxunk xxmaj xxunk - xxmaj xxunk and xxmaj hung xxmaj shi - xxmaj xxunk puts them in a "" xxmaj raiders of the xxmaj lost xxmaj ark "" setting . xxmaj however , these are not the light - hearted comedic xxunk that you might see in a xxmaj jet xxmaj li movie . xxmaj instead these guys"
4,"xxup shallow xxup grave begins with either a tribute or a rip off of the shower scene in xxup psycho . ( i 'm leaning toward rip off . ) xxmaj after that it gets worse and then surprisingly gets better , almost to the point of being original . xxmaj bad acting and amateurish directing xxunk down a fairly interesting little story , but the film already surpasses many"


In [12]:
data_clas.show_batch()

text,target
"xxbos xxmaj raising xxmaj victor xxmaj vargas : a xxmaj review \n \n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of xxunk . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj",negative
"xxbos xxup the xxup shop xxup around xxup the xxup corner is one of the sweetest and most feel - good romantic comedies ever made . xxmaj there 's just no getting around that , and it 's hard to actually put one 's feeling for this film into words . xxmaj it 's not one of those films that tries too hard , nor does it come up with",positive
"xxbos xxmaj now that xxmaj che(2008 ) has finished its relatively short xxmaj australian cinema run ( extremely limited xxunk screen in xxmaj sydney , after xxunk ) , i can xxunk join both xxunk of "" xxmaj at xxmaj the xxmaj movies "" in taking xxmaj steven xxmaj soderbergh to task . \n \n xxmaj it 's usually satisfying to watch a film director change his style /",negative
"xxbos xxmaj this film sat on my xxmaj tivo for weeks before i watched it . i dreaded a self - indulgent xxunk flick about relationships gone bad . i was wrong ; this was an xxunk xxunk into the screwed - up xxunk of xxmaj new xxmaj yorkers . \n \n xxmaj the format is the same as xxmaj max xxmaj xxunk ' "" xxmaj la xxmaj ronde",positive
"xxbos xxmaj many neglect that this is n't just a classic due to the fact that it 's the first xxup 3d game , or even the first xxunk - up . xxmaj it 's also one of the first stealth games , one of the xxunk definitely the first ) truly claustrophobic games , and just a pretty well - rounded gaming experience in general . xxmaj with graphics",positive


### Fine-tuning a language model 

In [13]:
torch.cuda.set_device(0)

In [14]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.352776,3.894770,0.285619,00:44


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,4.031151,3.937155,0.279480,00:49
1,3.698862,3.897959,0.284550,00:50


In [ ]:
learn.predict("This is a review about", n_words=10)

In [ ]:
learn.save('ft')
learn.save_encoder('ft_enc')

### Building a classifier 

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5).to_fp16()
learn.load_encoder('ft_enc')

In [ ]:
learn.fit_one_cycle(1, 1e-2)

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

In [ ]:
learn.predict("This was a great moive!")